# POS 8

The 8th try for the POS tagging

Brown is an annotated corpus for POS tags. A sample is available in the NLTK python library which contains a lot of corpora that can be used to train and test some NLP models.

In [1]:
# Import nltk and download the needed packages
import sklearn as sk
import pandas as pd
import re 
import string
import nltk
nltk.download('brown')
nltk.download('universal_tagset')
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
nltk.download("stopwords")

#import the created fucntions files 
import functions as f
import fakenews_utilities as fns

from nltk.corpus import stopwords
from nltk.corpus import brown
from collections import Counter

[nltk_data] Downloading package brown to
[nltk_data]     /Users/shaniaspierings/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/shaniaspierings/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shaniaspierings/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shaniaspierings/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shaniaspierings/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
punct_regex = re.compile('[%s]' % re.escape(string.punctuation))

df = pd.read_csv("tweets_labeled.csv", engine='python') 
df_test = df[0:50]
#df_test = fns.wash_pandas_str(test_df)
df_test.head(10)

,tweet_id,text,label
0,1161040537207463936,'RT @SenJeffMerkley: The Endangered Species Ac...,1
1,1176360756239118342,'RT @LindseyGrahamSC: Interesting concept -- i...,1
2,1099036648573145088,'RT @RealJamesWoods: #BuildTheWall #DeportThem...,0
3,1092915693203480577,'RT @PatriotJackiB: Why would the MEXICAN GOV’...,0
4,1149038450668187654,'RT @TheOnion: Sweden Announces Plan To Get 10...,0
5,1175456815674343424,'RT @kcResister: Warren: 'Congress is complici...,1
6,1180809117310623744,'RT @ShimonPro: A dozen current and former sta...,1
7,1179840318935576578,'RT @samstein: This is a bombshell that isn’t ...,1
8,1081722778125062144,'Planned Parenthood Erects Billboards Urging W...,0
9,1158761795739217921,'https://t.co/MvrznF1fWVWhoever obstructing th...,1


# The POS counting

In [3]:
# Some of the sources, might come in handy lateron
# Part of the source code: https://www.guru99.com/counting-pos-tags-nltk.html
# Other source: https://stackoverflow.com/questions/44088038/create-sentence-row-to-pos-tags-counts-column-matrix-from-a-dataframe
# https://stackoverflow.com/questions/54112105/typeerror-append-missing-1-required-positional-argument-other


# The library brown corpus consists of multiple tags that are used for the part of speech taggging. 
# This function collects the different tags that are included in the library.
def library_tags():
    brown_news_tagged = brown.tagged_words(categories='news', tagset='universal')
    tag_fd = nltk.FreqDist(tag for (word, tag) in brown_news_tagged)
    return tag_fd

# This function tokenizes the text and provides each token with a POS tag. 
def token_tag(x):
    tokens = nltk.word_tokenize(x)
    tags = nltk.pos_tag(tokens, tagset="universal") # tags contains of a list per row 
    return tags

# This function will be used to collects all the possible tags. 
# Compare the possible tags to the mentioned tags in token_tag. 
# If the possbile tag is not mentioned a '0' is added to the tag. 
def pos_to_possible_tags(counter, keys_to_add):
    for k in keys_to_add:
        counter[k] = counter.get(k, 0)
    return counter

# For the following function to work you need to work with a clean text.
# This function combines the above mentioned functions and provides a number along with a POS-tag.
def pos_numbers():
    tokens = df_test.loc[:, "text"].apply(token_tag)
    pos_counts = tokens.apply(lambda x: Counter(tag for word,  tag in x))
    pos_counts = pos_counts.apply(lambda x: pos_to_possible_tags(x, library_tags()))
    df_test["pos_numbers"] = pos_counts.apply(lambda x: [count for tag, count in sorted(x.most_common())])
    
    pos_df = pd.DataFrame(df_test['pos_numbers'].tolist())
    pos_df.columns = sorted(library_tags())
    return pos_df


In [ ]:
# The pos_numbers() function creates a new dataframe. Therefore, the pos_numbers() need to be merged to the existing dataframe.
# The merge of the dataframes is based on the index. 
df_test = pd.merge(df_test, pos_numbers(), left_index=True, right_index=True)

In [19]:
df_test.head(20)

,tweet_id,text,label,pos_numbers,.,ADJ,ADP,ADV,CONJ,DET,NOUN,NUM,PRON,PRT,VERB,X
0,1161040537207463936,The Endangered Species Act saved the bald eagl...,1,"[1, 0, 2, 0, 1, 4, 9, 0, 0, 1, 4, 0]",1,0,2,0,1,4,9,0,0,1,4,0
1,1176360756239118342,"Interesting concept -- impeach first, find fac...",1,"[4, 1, 1, 3, 0, 1, 8, 0, 0, 0, 3, 0]",4,1,1,3,0,1,8,0,0,0,3,0
2,1099036648573145088,#BuildTheWall #DeportThemAll,0,"[2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]",2,0,0,0,0,0,2,0,0,0,0,0
3,1092915693203480577,Why would the MEXICAN GOV’T fund this? Who are...,0,"[2, 0, 2, 1, 0, 2, 6, 0, 2, 0, 2, 0]",2,0,2,1,0,2,6,0,2,0,2,0
4,1149038450668187654,Sweden Announces Plan To Get 100% Of Energy Fr...,0,"[0, 0, 3, 0, 0, 0, 10, 2, 0, 1, 1, 0]",0,0,3,0,0,0,10,2,0,1,1,0
5,1175456815674343424,Warren: Congress is complicit by failing to st...,1,"[1, 1, 2, 0, 0, 0, 4, 0, 0, 1, 4, 0]",1,1,2,0,0,0,4,0,0,1,4,0
6,1180809117310623744,A dozen current and former staff from State De...,1,"[0, 3, 2, 0, 1, 2, 7, 0, 0, 0, 3, 0]",0,3,2,0,1,2,7,0,0,0,3,0
7,1179840318935576578,This is a bombshell that isn’t getting much no...,1,"[1, 4, 4, 0, 1, 4, 6, 0, 1, 0, 6, 0]",1,4,4,0,1,4,6,0,1,0,6,0
8,1081722778125062144,Planned Parenthood Erects Billboards Urging Wo...,0,"[1, 0, 0, 0, 0, 0, 8, 0, 0, 1, 1, 0]",1,0,0,0,0,0,8,0,0,1,1,0
9,1158761795739217921,obstructing the prosperity of our country MUS...,1,"[0, 0, 1, 0, 1, 1, 3, 0, 1, 0, 5, 0]",0,0,1,0,1,1,3,0,1,0,5,0


# Calculate the correlation

In [20]:
df_subset = df_test[["label", ".", "ADJ", "ADP", "CONJ", "DET", "NOUN", "NUM", "PRON", "PRT", "VERB", "X"]]
corr = df_subset.corr()
corr

,label,.,ADJ,ADP,CONJ,DET,NOUN,NUM,PRON,PRT,VERB,X
label,1.000000,-0.095329,0.271206,0.248027,0.186019,0.312342,-0.132125,-0.003920,0.282842,0.164197,0.608880,-0.146508
.,-0.095329,1.000000,0.130892,-0.066116,-0.126609,0.029935,0.128109,-0.027051,0.073445,-0.057940,-0.029869,0.289917
ADJ,0.271206,0.130892,1.000000,0.562538,0.150039,0.345727,-0.130793,-0.134905,0.272105,-0.027037,0.411048,0.421464
ADP,0.248027,-0.066116,0.562538,1.000000,-0.021224,0.281570,0.095460,0.138349,0.379949,-0.012111,0.301185,0.450994
CONJ,0.186019,-0.126609,0.150039,-0.021224,1.000000,0.226354,-0.089242,-0.153406,0.177137,-0.024340,0.351555,-0.112420
DET,0.312342,0.029935,0.345727,0.281570,0.226354,1.000000,0.037553,-0.114480,0.476361,0.058124,0.497615,-0.167789
NOUN,-0.132125,0.128109,-0.130793,0.095460,-0.089242,0.037553,1.000000,-0.043478,-0.205449,0.004038,-0.205529,-0.051290
NUM,-0.003920,-0.027051,-0.134905,0.138349,-0.153406,-0.114480,-0.043478,1.000000,-0.213075,0.377506,-0.142332,-0.056857
PRON,0.282842,0.073445,0.272105,0.379949,0.177137,0.476361,-0.205449,-0.213075,1.000000,0.114739,0.661589,0.080440
PRT,0.164197,-0.057940,-0.027037,-0.012111,-0.024340,0.058124,0.004038,0.377506,0.114739,1.000000,0.334857,-0.144338
